In [1]:
import pandas as pd,seaborn as sns,matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import json
import requests
from bs4 import BeautifulSoup
from modules.functions import get_schedule,create_home_and_away_simple_dataframe

In [2]:
get_schedule("Purdue")

,DATE,OPPONENT,GAME_ID
2,"Tue, Nov 8",Milwaukee,401483405
3,"Fri, Nov 11",Austin Peay,401483406
4,"Tue, Nov 15",Marquette,401479696
5,"Thu, Nov 24",West Virginia,401483331
6,"Fri, Nov 25",Gonzaga,401486748
7,"Sun, Nov 27",Duke,401486751
9,"Wed, Nov 30",@ Florida State,401479679
10,"Sun, Dec 4",Minnesota,401484836
11,"Wed, Dec 7",Hofstra,401483407
12,"Sat, Dec 10",@ Nebraska,401484843


In [43]:
create_home_and_away_simple_dataframe(401486751,disp=True)

Player Position MIN     FG   3PT     FT OREB DREB REB AST STL  \
Duke 1     M. Mitchell        F  25    1-6   0-2    0-0    1    2   3   1   0   
     2   K. Filipowski        C  31   5-11   2-4    2-3    2    3   5   0   1   
     3    D. Lively II        C  22    0-3   0-2    0-0    3    2   5   0   2   
     4      T. Proctor        G  36   7-13   0-4    2-2    1    4   5   3   0   
     5        J. Roach        G  37   5-13   0-3    4-6    0    3   3   2   2   
     7    D. Whitehead        F  16    2-7   0-2    2-2    0    1   1   0   0   
     8       C. Reeves        C   0    0-0   0-0    0-0    0    0   0   0   0   
     9        R. Young        C  15    1-3   0-0    2-2    3    1   4   0   0   
     10      J. Blakes        G   9    0-2   0-2    0-2    0    2   2   1   0   
     11   J. Grandison        G   9    0-0   0-0    0-0    0    0   0   1   0   
     12           Team               21-58  2-19  12-17   11   20  31   8   5   

        BLK  TO  PF PTS  
Duke 1    0   2   0   2  
     2    1   2   5  14  
     3    2   1   5   0  
     4    0   1   1  16  
     5    0   2   2  14  
     7    0   1   1   6  
     8    0   0   0   0  
     9    0   0   3   4  
     10   0   0   0   0  
     11   0   1   2   0  
     12   3  11  19  56

Player Position MIN     FG   3PT     FT OREB DREB REB AST  \
Purdue 1         M. Gillis        F  14    2-3   2-2    2-2    1    0   1   0   
       2           Z. Edey        C  32   7-13   0-0    7-8    5    7  12   3   
       3          B. Smith        G  24    2-5   0-2    0-0    1    7   8   2   
       4          F. Loyer        G  34   6-14   4-8    2-2    0    2   2   0   
       5         E. Morton        G  26    1-5   0-3    0-0    1    5   6   6   
       7   T. Kaufman-Renn        F  10    1-3   0-0    0-0    1    0   1   0   
       8        B. Waddell        F   6    1-2   0-0    0-0    0    0   0   1   
       9          C. Furst        F  24    4-6   0-0    3-5    5    5  10   2   
       10        B. Newman        G  14    1-4   1-3    0-0    0    1   1   2   
       11   D. Jenkins Jr.        G  16    1-3   0-0    2-2    0    0   0   1   
       12             Team               26-58  7-18  16-19   14   28  42  17   

          STL BLK  TO  PF PTS  
Purdue 1    0   0   3   1   8  
       2    0   0   0   1  21  
       3    3   0   3   3   4  
       4    2   0   1   1  18  
       5    2   1   1   2   2  
       7    0   1   1   2   2  
       8    0   0   0   1   2  
       9    0   1   0   2  11  
       10   0   0   1   2   3  
       11   0   0   1   1   4  
       12   7   3  11  16  75